In [76]:
code = {
    '23501': 'sig_tot',
    '23522': 'sig_ion',
    '23525': 'sig_scat_la',
    '23526': 'sig_scat',
    '23527': 'sig_brem',
    '23528': 'sig_exc',
    '23534': 'sig_ion_K',
    '23535': 'sig_ion_L1',
    '23536': 'sig_ion_L2',
    '23537': 'sig_ion_L3',
    '23538': 'sig_ion_M1',
    '23539': 'sig_ion_M2',
    '23540': 'sig_ion_M3',
    '23541': 'sig_ion_M4',
    '23542': 'sig_ion_M5',
    '23543': 'sig_ion_N1',
    '23544': 'sig_ion_N2',
    '23545': 'sig_ion_N3',
    '23546': 'sig_ion_N4',
    '23547': 'sig_ion_N5',
    '23548': 'sig_ion_N6',
    '23549': 'sig_ion_N7',
    '23550': 'sig_ion_O1',
    '23551': 'sig_ion_O2',
    '23552': 'sig_ion_O3',
    '23553': 'sig_ion_O4',
    '23554': 'sig_ion_O5',
    '23555': 'sig_ion_O6',
    '23556': 'sig_ion_O7',
    '23559': 'sig_ion_P1',
    '23560': 'sig_ion_P2',
    '23561': 'sig_ion_P3',
    '23562': 'sig_ion_P4',
    '23563': 'sig_ion_P5',
    '23570': 'sig_ion_Q1',
    '26525': 'th_scat',
    '26527': 'spec_brem',
    '26528': 'spec_exc',
    '26534': 'spec_ion_K',
    '26535': 'spec_ion_L1',
    '26536': 'spec_ion_L2',
    '26537': 'spec_ion_L3',
    '26538': 'spec_ion_M1',
    '26539': 'spec_ion_M2',
    '26540': 'spec_ion_M3',
    '26541': 'spec_ion_M4',
    '26542': 'spec_ion_M5',
    '26543': 'spec_ion_N1',
    '26544': 'spec_ion_N2',
    '26545': 'spec_ion_N3',
    '26546': 'spec_ion_N4',
    '26547': 'spec_ion_N5',
    '26548': 'spec_ion_N6',
    '26549': 'spec_ion_N7',
    '26550': 'spec_ion_O1',
    '26551': 'spec_ion_O2',
    '26552': 'spec_ion_O3',
    '26553': 'spec_ion_O4',
    '26554': 'spec_ion_O5',
    '26555': 'spec_ion_O6',
    '26556': 'spec_ion_O7',
    '26559': 'spec_ion_P1',
    '26560': 'spec_ion_P2',
    '26561': 'spec_ion_P3',
    '26562': 'spec_ion_P4',
    '26563': 'spec_ion_P5',
    '26570': 'spec_ion_Q1'
}


In [77]:
import os
import numpy as np
from types import SimpleNamespace
import const


In [78]:
elements = SimpleNamespace(**np.load('elements.npz'))


In [79]:
with open('../EPICS/EEDL.txt', 'r') as f:
    lines_all = f.readlines()


In [157]:
nelem = 99
ndtype = len(code)


In [158]:
lines_sorted = np.empty((nelem, ndtype), dtype=object)
for i in range(nelem):
    for j in range(ndtype):
        lines_sorted[i, j] = []

for line in lines_all:
     
    if line[66:68] == '  ': continue
    if line[70:75] not in code.keys(): continue
    
    Zelem = int(line[66:68])
    dtype_idx = list(code.keys()).index(line[70:75])
    line_data = [line[0:11], line[11:22], line[22:33], line[33:44], line[44:55], line[55:66]]
    line_data = np.array([float(val) if val != '           ' else np.nan for val in line_data])
    lines_sorted[Zelem-1, dtype_idx].append(line_data)
    
for i in range(nelem):
    for j in range(ndtype):
        lines_sorted[i, j] = np.array(lines_sorted[i, j])
        

In [159]:
data = {}

data['Z'] = []
data['symbol'] = []
data['data'] = []

for i in range(nelem):
    
    Zelem = i+1
    data_elem = {'ion_list': [], 'sig_ion_list': [], 'spec_ion_list': []}
    
    for j in range(ndtype):
        
        lines = lines_sorted[i, j]
        if lines.size == 0: continue
            
        dtype_name = list(code.values())[j]
        data_dtype = {}
        
        if dtype_name[:3] == 'sig':
            
            ener, sig = lines[3:].flatten().reshape(-1, 2).T
            ener, sig = list(ener[~np.isnan(ener)][1:]), list(sig[~np.isnan(sig)][1:]*const.ba)
            
            data_dtype['ener'] = ener
            data_dtype['sig'] = sig
            
            if dtype_name[:7] == 'sig_ion' and len(dtype_name) > 7: 
                ener_bind = lines[1, 0]
                data_dtype['ener_bind'] = ener_bind
            
        if dtype_name == 'th_scat':
            
            idx_line_new_ener = np.argwhere(lines[8:, 0]==0).flatten()
            ener_list = list(lines[8:, 1][idx_line_new_ener])
            
            data_dtype['ener'] = ener_list
            data_dtype['cos_th'] = []
            data_dtype['cos_th_dist'] = []

            for k, ener in enumerate(ener_list):

                idx_line_min = idx_line_new_ener[k]+1
                idx_line_max = idx_line_new_ener[k+1] if k<len(ener_list)-1 else -1
                cos_th_scat, cos_th_pdf = lines[8:, :][idx_line_min:idx_line_max].flatten().reshape(-1, 2).T
                cos_th_scat, cos_th_pdf = list(cos_th_scat[~np.isnan(cos_th_scat)]), list(cos_th_pdf[~np.isnan(cos_th_pdf)])
                
                data_dtype['cos_th'].append(cos_th_scat)
                data_dtype['cos_th_dist'].append(cos_th_pdf)
                
        if dtype_name == 'spec_exc':
            
            ener, ener_loss = lines[6:].flatten().reshape(-1, 2).T
            ener, ener_loss = list(ener[~np.isnan(ener)]), list(ener_loss[~np.isnan(ener_loss)])
            
            data_dtype['ener'] = list(ener)
            data_dtype['ener_loss'] = list(ener_loss)
            
        if dtype_name == 'spec_brem':
            
            idx_line_elec = np.argwhere(lines[6:, 0]==0).flatten()[-1]+8
            ener, ener_loss = lines[idx_line_elec:].flatten().reshape(-1, 2).T
            ener, ener_loss = list(ener[~np.isnan(ener)]), list(ener_loss[~np.isnan(ener_loss)])
            
            data_dtype['ener'] = ener
            data_dtype['ener_loss'] = ener_loss
            
        if dtype_name[:8] == 'spec_ion':
            
            idx_line_new_ener = np.argwhere(lines[6:, 0]==0).flatten()
            ener_list = list(lines[6:, 1][idx_line_new_ener])
            
            data_dtype['ener'] = ener_list
            data_dtype['ener_loss'] = []
            data_dtype['ener_loss_dist'] = []

            for k, ener in enumerate(ener_list):

                idx_line_min = idx_line_new_ener[k]+1
                idx_line_max = idx_line_new_ener[k+1] if k<len(ener_list)-1 else -1
                ener_loss, ener_loss_pdf = lines[6:, :][idx_line_min:idx_line_max].flatten().reshape(-1, 2).T
                ener_loss, ener_loss_pdf = list(ener_loss[~np.isnan(ener_loss)] + ener_bind), list(ener_loss_pdf[~np.isnan(ener_loss_pdf)])
                
                data_dtype['ener_loss'].append(ener_loss)
                data_dtype['ener_loss_dist'].append(ener_loss_pdf)
                
        if dtype_name[:7] == 'sig_ion' and len(dtype_name) > 7: 
            data_elem['ion_list'].append(dtype_name[8:])
            data_elem['sig_ion_list'].append(data_dtype)
        elif dtype_name[:8] == 'spec_ion':
            data_elem['spec_ion_list'].append(data_dtype)   
        else:
            data_elem[dtype_name] = data_dtype
        
    data['Z'].append(Zelem)
    data['symbol'].append(elements.symbol[i])
    data['data'].append(data_elem)
            

In [148]:
import json
with open('/Users/za9132/C++/thunderstorm/resources/EEDL.json', 'w') as f:
    json.dump(data, f)
